In [31]:
import ee
import geemap
import numpy as np
import sys
import geopandas as gpd
sys.path.insert(1, '/Users/gopal/Projects/ml/downloadGEErasters')
sys.path.insert(1, '/Users/gopal/Projects/sasia/src/proc/')
import rs
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from gee_landsat_prep_functions import prep_tm5_ic, prep_etm7_ic, prep_oli8_ic, prep_landsat_clouds

In [10]:
ee.Initialize()

In [26]:
ee_geometry = ee.FeatureCollection('users/gopalpenny/arkavathytanks/ArkavatiSubbasins')
tg_halli = ee_geometry.filter(ee.Filter.eq('Subcatch', 'TG_Halli'))
tg_halli.aggregate_array('Subcatch').getInfo()

['TG_Halli']

In [64]:

tm5 = prep_tm5_ic(ee_geometry).filterDate('2014-11-01','2015-02-01')
etm7 = prep_etm7_ic(ee_geometry).filterDate('2014-11-01','2015-02-01')
oli8 = prep_oli8_ic(tg_halli).filterDate('2014-11-01','2015-02-01')

landsat = tm5.merge(etm7).merge(oli8)

In [68]:
landsat = landsat #\
  # .filterDate('2014-11-01','2015-02-01') #\
  # .filter(ee.Filter.inList('DATE_ACQUIRED', ee.List(['2014-11-10','2015-01-13'])))

oli8.aggregate_array('DATE_ACQUIRED').getInfo()

['2014-11-10',
 '2014-11-26',
 '2014-12-12',
 '2014-12-28',
 '2015-01-13',
 '2015-01-29']

In [39]:
samp_pts = gpd.read_file('/Users/gopal/Library/CloudStorage/Dropbox/Berkeley/Research/MyResearch/2014ArkavathyIndia/Arkavathy shp file/ACCUWA_data/landuse/Tg_halli/Landuse_village_datapoints_15_01_2015.shp')
# samp_pts_ee = geemap.geopandas_to_ee(samp_pts)

oli8.select([0]).projection()

In [47]:
# print(rs.get_pixel_ts_allbands.__doc__)
ts = rs.get_pixel_ts_allbands(samp_pts_ee, oli8, 'DATE_ACQUIRED', scale = 30)
# import inspect
# print(inspect.getsource(rs.get_pixel_ts_allbands))

In [79]:
ts_gpd = geemap.ee_to_geopandas(ts)
ts_gpd.to_file('../../data/format/get_landsat_for_lulc/lulc_oli8.shp')

In [74]:
m = geemap.Map()
m.addLayer(oli8.filterDate('2014-11-01','2014-12-01').first(),{'bands':['nir','red','green'], 'max': 30000},'oli8 2014')
m.addLayer(oli8.filterDate('2015-01-01','2015-12-01').first(),{'bands':['nir','red','green'], 'max': 30000},'oli8 2015')
m.addLayer(ee.Image().paint(ee_geometry,1,1),{'palette':'white'},'ark')
m.addLayerControl()
m.centerObject(ee_geometry, 10)
m

Map(center=[12.84852761341192, 77.44092789490166], controls=(WidgetControl(options=['position', 'transparent_b…